In [10]:
import pandas as pd
import seaborn as sns
import itertools
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import Counter 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


Code to generate synthetic data of all possible scenarios of Harry kanes next penalty

In [11]:
# Set parameters
isCruciality = [0,1]
IsFriendlyLeagueGame = [0,1]
WasThePlayerSubstitutedOn = [0,1]
PenaltiesTakenInGame = [0,1]

# Previous walkin time for penalty
PrevWalkIn = [507]


CardsInGame = [0, 1, 2]
PenaltyTime = range(1, 91)  # From 1 to 90
GoalDifferenceDuringGoal = [-1, 0, 1]
WalkDuration = range(300, 701, 50)  # From 350 to 800, step 25
FanAdvantage = [0,1,2]
DidPlayerWinPenalty = [0, 1]
isPlayerBooked = [0, 1]
GoalsInGame = [0, 1]
AssistsInGame = [0, 1]

# Create a function to determine if it's the first half based on PenaltyTime
def isFirsthalf(penalty_time):
    return 1 if penalty_time < 46 else 0

# Create a function to determine if it's extra time based on PenaltyTime
def isExtratime(penalty_time):
    return 1 if penalty_time in [45, 90] else 0

# Generate all possible combinations
combinations = list(itertools.product(isCruciality, IsFriendlyLeagueGame, WasThePlayerSubstitutedOn,
                                      PenaltiesTakenInGame, PrevWalkIn, CardsInGame, PenaltyTime,
                                      GoalDifferenceDuringGoal, WalkDuration, FanAdvantage,
                                      DidPlayerWinPenalty, isPlayerBooked, GoalsInGame, AssistsInGame))

# Build the dataframe from the combinations and apply isFirsthalf and isExtratime rules
df_combinations = pd.DataFrame(combinations, columns=[
    'isCruciality', 'IsFriendlyLeagueGame', 'WasThePlayerSubstitutedOn', 'PenaltiesTakenInGame', 'PrevWalkIn',
    'CardsInGame', 'PenaltyTime', 'GoalDifferenceDuringGoal', 'WalkDuration', 'FanAdvantage',
    'DidPlayerWinPenalty', 'isPlayerBooked', 'GoalsInGame', 'AssistsInGame'])


df_combinations['isFirsthalf'] = df_combinations['PenaltyTime'].apply(isFirsthalf)
df_combinations['isExtratime'] = df_combinations['PenaltyTime'].apply(isExtratime)
df_combinations['NoOfMinPlayedByThePlayer'] = df_combinations['PenaltyTime']

# HK previous bucket class
df_combinations['PrevBucket'] = 1

#Current state of HK probabilities
df_combinations['Prob_0'] = 0.166667
df_combinations['Prob_1'] = 0.559524
df_combinations['Prob_2'] = 0.273810

#bias column
df['bias'] = 1

og = df_combinations.copy()
df_combinations.shape

(5598720, 21)

Normalization

In [12]:
# Load the saved scaler
scaler = joblib.load('./models/min_max_scaler.pkl')

In [13]:
# Columns to normalize
cols_to_normalize = ['NoOfMinPlayedByThePlayer', 'PenaltyTime','WalkDuration','PrevWalkIn','Prob_2','Prob_1','Prob_0']

# Apply scaling only to specific columns
df_combinations[cols_to_normalize] = scaler.fit_transform(df_combinations[cols_to_normalize])

In [14]:
desired_col_order = ['isCruciality', 'IsFriendlyLeagueGame', 'CardsInGame', 'PenaltyTime',
                     'isFirsthalf', 'isExtratime', 'GoalDifferenceDuringGoal',
                     'WalkDuration', 'FanAdvantage', 'WasThePlayerSubstitutedOn',
                     'NoOfMinPlayedByThePlayer', 'DidPlayerWinPenalty', 'isPlayerBooked',
                     'GoalsInGame', 'AssistsInGame', 'PenaltiesTakenInGame', 'PrevBucket',
                     'PrevWalkIn', 'Prob_0', 'Prob_1', 'Prob_2', 'bias']

# Reorder the columns in df_combinations
df_combinations = df_combinations[desired_col_order]

Load models and predict

In [15]:
# Load the KNN with n=7
model_7n = joblib.load('./models/model_7n.pkl')

# Load the KNN with n=7
model_8n = joblib.load('./models/model_8n.pkl')

In [18]:
# Get class probabilities for all test instances
probabilities = model_7n.predict_proba(df_combinations)

# Create a DataFrame with actual values and probabilities
results_df = og.copy()
results_df['Class 0 Prob - 7n'] = [prob[0] for prob in probabilities]
results_df['Class 1 Prob - 7n'] = [prob[1] for prob in probabilities]
results_df['Class 2 Prob - 7n'] = [prob[2] for prob in probabilities]

In [19]:
# Get class probabilities for all test instances
probabilities = model_8n.predict_proba(df_combinations)

# Create a DataFrame with actual values and probabilities
results_df['Class 0 Prob - 8n'] = [prob[0] for prob in probabilities]
results_df['Class 1 Prob - 8n'] = [prob[1] for prob in probabilities]
results_df['Class 2 Prob - 8n'] = [prob[2] for prob in probabilities]

Export data

In [ ]:
results_df.to_csv('HarryKaneNextPenaltyPredictions.csv')